In [1]:
import tetris_engine as tr
import numpy as np
np.set_printoptions(linewidth=300)
from IPython.display import clear_output
import random as rnd
import torch
import torch.nn as nn
import random
import time
# import os, sys
# os.environ["SDL_VIDEODRIVER"] = "dummy"

pygame 2.1.2 (SDL 2.0.16, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
env = tr.GameState()

In [9]:
"""
@author: Viet Nguyen <nhviet1009@gmail.com>
""" 
import torch.nn as nn

class DeepQNetwork(nn.Module):
    def __init__(self):
        super(DeepQNetwork, self).__init__()

        self.conv1 = nn.Sequential(nn.Linear(5, 64), nn.LeakyReLU(inplace=True))
        self.conv2 = nn.Sequential(nn.Linear(64, 64), nn.LeakyReLU(inplace=True))
        self.conv3 = nn.Sequential(nn.Linear(64, 64), nn.LeakyReLU(inplace=True))
        self.conv4 = nn.Sequential(nn.Linear(64, 64), nn.LeakyReLU(inplace=True))
        self.conv5 = nn.Sequential(nn.Linear(64, 1))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)

        return x

In [3]:
class LinearBlock(nn.Module):
    """
    Linear layer with ReLU and BatchNorm
    """
    def __init__(self, input_prev, embed_dim):
        super(LinearBlock, self).__init__()
        self.fc = nn.Linear(input_prev, embed_dim)
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm1d(embed_dim)

    def forward(self, inputs):
        x = inputs
        x = self.fc(x)
        x = self.relu(x)
        x = self.bn(x)
        return x


class ResidualBlock(nn.Module):
    """
    Residual block with two linear layers
    """
    def __init__(self, embed_dim):
        super(ResidualBlock, self).__init__()
        self.linearblock_1 = LinearBlock(embed_dim, embed_dim)
        self.linearblock_2 = LinearBlock(embed_dim, embed_dim)

    def forward(self, inputs):
        x = inputs
        x = self.linearblock_1(x)
        x = self.linearblock_2(x)
        x += inputs # skip-connection
        return x


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.one_hot = nn.functional.one_hot
        self.Stack = nn.Sequential(
            LinearBlock(6, 32),
            LinearBlock(32, 128),
            LinearBlock(128, 500),
            ResidualBlock(500),
            ResidualBlock(500),
            ResidualBlock(500),
            ResidualBlock(500),
        )
        self.Prediction = nn.Linear(500, 1)

    def forward(self, inputs):
        x = inputs
        # x = self.one_hot(x, num_classes=6).to(torch.float).reshape(-1, 324)
        x = self.Stack(x)
        logits = self.Prediction(x)
        return logits

In [7]:

model = torch.load('trained_models/tetris_4500')

counter = 0
# for seed in range(10000):
counter+=1
random.seed(4298)
done = False
reward = 0
reward_check = 0
env.reinit()

next_steps = env.get_next_states()
next_actions, next_states = zip(*next_steps.items())
next_states = torch.stack(next_states)
if torch.cuda.is_available():
    next_states = next_states.cuda()
model.eval()
predictions = model(next_states)[:, 0]

while True:
    if reward != reward_check:
        with torch.no_grad():
            next_steps = env.get_next_states()
            next_actions, next_states = zip(*next_steps.items())
            next_states = torch.stack(next_states)
            if torch.cuda.is_available():
                next_states = next_states.cuda()
            model.eval()
            predictions = model(next_states)[:, 0]


    index = torch.argmax(predictions).item()

    action = next_actions[index]

    reverse_x = env.fallingPiece['x']
    reverse_rot = env.fallingPiece['rotation']
    env.fallingPiece['x'] = action[0]
    env.fallingPiece['rotation'] = action[1]

    if not env.isValidPosition():
        env.fallingPiece['x'] = reverse_x
        env.fallingPiece['rotation'] = reverse_rot

    reward_check = reward
    reward, done = env.frame_step([0,0,0,0,0,0])[1:3]
        # if env.lines > 150:
        #     print('This is the seed: ', seed)
        #     print('Lines: ', env.lines)
        # if done:
        #     break
    

    
    

KeyboardInterrupt: 

In [8]:
model = torch.load('trained_models/res_arq_3000_morelr/tetris_res_arq_3000_morelr_2')
counter = 0
# for seed in range(10000):
counter+=1
random.seed(103)
done = False
reward = 0
reward_check = 0
env.reinit()

next_steps = env.get_next_states()
next_actions, next_states = zip(*next_steps.items())
next_states = torch.stack(next_states)
if torch.cuda.is_available():
    next_states = next_states.cuda()
model.eval()
predictions = model(next_states)[:, 0]

while True:
    if reward != reward_check:
        with torch.no_grad():
            next_steps = env.get_next_states()
            next_actions, next_states = zip(*next_steps.items())
            next_states = torch.stack(next_states)
            if torch.cuda.is_available():
                next_states = next_states.cuda()
            model.eval()
            predictions = model(next_states)[:, 0]


    index = torch.argmax(predictions).item()

    action = next_actions[index]

    reverse_x = env.fallingPiece['x']
    reverse_rot = env.fallingPiece['rotation']
    env.fallingPiece['x'] = action[0]
    env.fallingPiece['rotation'] = action[1]

    if not env.isValidPosition():
        env.fallingPiece['x'] = reverse_x
        env.fallingPiece['rotation'] = reverse_rot

    reward_check = reward
    reward, done = env.frame_step([0,0,0,0,0,0])[1:3]
    # time.sleep(0.025)
    # if env.lines > 1000:
    #     print('This is the seed: ', seed)
    #     print('Lines: ', env.lines)
    # if done:
    #     break
    

    
    

KeyboardInterrupt: 

In [ ]:
4500(4298)253

In [ ]:
5000(2269)174

In [ ]:
2473

In [ ]:
3969